In [1]:
#Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re

##**Obtain Data**

In [2]:
df = pd.read_csv('../input/Basic_Stats.csv')

##**Scrubbing The Data**

In [3]:
df.dtypes

In [4]:
#I will rename the weight and height column so i can work on it easier
df.rename(columns={'Height (inches)':'Height',
                  'Weight (lbs)':'Weight'}, inplace = True)

In [5]:
#Let's check how many columns have a null value
df.isnull().sum()

It looks like Current Team and Position are the same. High school and High School Location are also very similar. 

In [6]:
df.dropna(subset=['Age','Birthday','Height','Weight'],inplace=True)
df.shape

In [7]:
#A quick analysis of Years Played and Current Team, and Current Status
print(df['Current Status'].value_counts())
#Separate the start and end dates for non null items in years played
df['start_year'] = df['Years Played'].astype(str).str[:4]
df['end_year'] = df['Years Played'].astype(str).str[7:]

df.Experience.value_counts()[:10]

In [8]:
df['Experience'] = df['Experience'].str.split(" ")
df['Experience'] = [i[0] for i in df['Experience']]

In [9]:
#Extract number from string
num = list()
for i in df['Experience']:
    result = re.sub('[^0-9]',"", i)
    if result == "":
        result == 0
    num.append(result)
df['Experience'] = num
#Replace blank values which were Rookies to 0
num = list()
for i in df['Experience']:
    if i == "":
        i = 0
    num.append(i)
df['Experience'] = num
#Make Experience column INT Data type
df.Experience = df.Experience.astype(int)

##**Explore The Data**

Lets see which team creates the most NFL players

In [10]:
top_20 = df.College.value_counts()[:20]

fig, ax = plt.subplots()
ax = sns.barplot(x = top_20.index, y = top_20.values)
ax.set_xticklabels(top_20.index, rotation=90)
plt.show()

It shows that USC produces the most NFL players. Most of these schools are big "college football" schools

Additionally, because we turned Experience into INT Series, we can see how the average career length of an NFL player

In [11]:
ax = sns.boxplot(df.Experience, orient = "h")
ax.set_xticks(np.arange(0,28,2))
plt.show()

##**Multivariable Analysis**
What interests me now is the average career length of players from top schools. My hypothesis is that they will be longer than the average length because they are from top schools

In [12]:
#First we will get the average career length of these top schools
df_group_col = df.groupby("College", as_index = False).mean()

In [13]:
top_20 = df_group_col[df_group_col['College'].isin(list(top_20.index))]
top_20_weight = top_20.Weight.mean()
avg_weight = df.Weight.mean()
print("Top 20 schools average weight = " + str(top_20_weight))
print("Average Weight of all schools = " + str(avg_weight))

There is a 2.1 difference in weight. 

One-Sample T-Test (Measuring Satisfaction Level)¶

A one-sample t-test checks whether a sample mean differs from the population mean. We are going to see if there is a significant difference in weight between top 20 schools that produced NFL players and population we have in this data set.

Null Hypothesis: (H0: pTS = pES) The null hypothesis would be that there is no difference in weight level between players in top 20 schools

Alternate Hypothesis: (HA: pTS != pES) The alternative hypothesis would be that there is a difference in satisfaction level between players in top 20 schools


In [14]:
from scipy import stats
stats.ttest_1samp(a = top_20.Weight,
                 popmean= df['Weight'].mean())

The p value is 0.002 which is smaller than the alpha of 0.05. With a 95% confidence interval, this sample provides us with enough evidence to reject the null hypothesis